In [5]:
import os
os.sys.path.append('/Users/atmavidyavirananda/Desktop/FIND IT 2022/findit_env/lib/python3.8/site-packages')

In [60]:
import pandas as pd
import numpy as np

# Models
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, AdaBoostRegressor
from sklearn.metrics import classification_report as report
from xgboost import XGBClassifier, plot_importance, XGBRegressor
# from lightgbm import LGBMRegressor

# Evaluations
from sklearn.metrics import confusion_matrix, mean_squared_error as mse
from sklearn.model_selection import KFold, cross_val_score, RepeatedKFold, cross_val_predict, StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [76]:
train_df = pd.read_csv('train.csv', sep="|")
test_df = pd.read_csv('predict-case.csv', sep="|")
sample_sub = pd.read_csv('sample_submission.csv', sep=",")

In [9]:
print('Train shape: ', train_df.shape)
print('Test shape: ', test_df.shape)

Train shape:  (31746, 15)
Test shape:  (3000, 14)


In [38]:
# not used due to URL warning
# train_df.to_excel('train.xlsx', index=False, engine='xlsxwriter')

# avoid URL warning
with pd.ExcelWriter('train.xlsx', options={'strings_to_urls': False}) as writer:
        train_df.to_excel(writer, 'Sheet1')

# Data Cleaning

In [10]:
numeric_cols = ['experience_level', 'company_process_time', 'company_size']
category_cols = ['location', 'career_level', 'education_level', 'job_function', 'company_industry']
other_cols = ['job_title', 'job_benefits', 'job_description']

In [11]:
# subset train
train_num = train_df.copy()[numeric_cols]
train_cat = train_df.copy()[category_cols]
train_other = train_df.copy()[other_cols]

# subset train
test_num = test_df.copy()[numeric_cols]
test_cat = test_df.copy()[category_cols]
test_other = test_df.copy()[other_cols]

## Numerical

In [12]:
## EXPERIENCE LEVEL

def f_experience(x):
    if (x != 'Lebih dari 20 Tahun' )&(pd.notna(x)):
        return int(x.split(' ')[0])
    elif (x == 'Lebih dari 20 Tahun'):
        return 20 # cap at 20
    else:
        return x

# transform
train_num['experience_level'] = train_num['experience_level'].map(f_experience)
test_num['experience_level'] = test_num['experience_level'].map(f_experience)
    
# print uniques
print('train exp_level uniques: ', sorted(train_num['experience_level'].unique()))
print('test exp_level uniques: ', sorted(test_num['experience_level'].unique()))

train exp_level uniques:  [1.0, 2.0, 3.0, 4.0, 5.0, nan, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 20.0]
test exp_level uniques:  [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, nan, 7.0, 10.0, 12.0, 15.0, 20.0]


In [13]:
## COMPANY PROCESS TIME

def f_process_time(x):
    if pd.notna(x):
        return int(x.split(' ')[0])
    else:
        return x
    
# transform
train_num['company_process_time'] = train_num['company_process_time'].map(f_process_time)
test_num['company_process_time'] = test_num['company_process_time'].map(f_process_time)

# print uniques
print('train process_time uniques: \n', sorted(train_num['company_process_time'].unique()))
print('test process_time uniques: \n', sorted(test_num['company_process_time'].unique()))

train process_time uniques: 
 [nan, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0]
test process_time uniques: 
 [nan, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0]


In [14]:
## COMPANY SIZE

def f_company_size(x):
    if x == 'Lebih dari 5000 pekerja':
        return (5000, 5000) # cap at 5000
    elif x == '1- 50 pekerja':
        return (1, 50)
    elif pd.notna(x):
        splitted = x.split(' ')
        return (int(splitted[0]), int(splitted[2]))
    else:
        return x
    
# transform
train_num['company_size_min'] = train_num['company_size'].map(f_company_size).map(lambda t: t[0] if pd.notna(t) else t)
train_num['company_size_max'] = train_num['company_size'].map(f_company_size).map(lambda t: t[1] if pd.notna(t) else t)
del train_num['company_size']

test_num['company_size_min'] = test_num['company_size'].map(f_company_size).map(lambda t: t[0] if pd.notna(t) else t)
test_num['company_size_max'] = test_num['company_size'].map(f_company_size).map(lambda t: t[1] if pd.notna(t) else t)
del test_num['company_size']

## Category

In [15]:
## LOCATION

In [16]:
## career_level
train_df.groupby('career_level').agg({'salary':np.mean})

,salary
career_level,
CEO/GM/Direktur/Manajer Senior,2.615918e+07
Lulusan baru/Pengalaman kerja kurang dari 1 tahun,4.411308e+06
Manajer/Asisten Manajer,1.293844e+07
Pegawai (non-manajemen & non-supervisor),5.550787e+06
Supervisor/Koordinator,7.614601e+06
Tidak Terspesifikasi,NaN


In [17]:
train_cat['career_level'].unique()

array(['Manajer/Asisten Manajer', 'Supervisor/Koordinator',
       'Pegawai (non-manajemen & non-supervisor)',
       'Lulusan baru/Pengalaman kerja kurang dari 1 tahun',
       'Tidak Terspesifikasi', 'CEO/GM/Direktur/Manajer Senior'],
      dtype=object)

In [18]:
career_level_dict = {'Manajer/Asisten Manajer': 4, 
                     'Supervisor/Koordinator': 3, 
                     'Pegawai (non-manajemen & non-supervisor)': 2, 
                     'Lulusan baru/Pengalaman kerja kurang dari 1 tahun': 1, 
                     'Tidak Terspesifikasi': 0, 
                     'CEO/GM/Direktur/Manajer Senior': 5}

# transform
train_cat['career_level'] = train_cat['career_level'].map(lambda x: career_level_dict[x])
test_cat['career_level'] = test_cat['career_level'].map(lambda x: career_level_dict[x])

#### education_level

In [19]:
## education_level (note: no NaNs in train nor test)d
l = []

a = list(train_cat['education_level'].unique())
b = list(test_cat['education_level'].unique())
a.extend(b)

for i in set(a):
    for j in i.split(','):
        l.append(j.strip().lower())
        
set(l)

{'d3 (diploma)',
 'd4 (diploma)',
 'diploma pascasarjana',
 'doktor (s3)',
 'gelar professional',
 'magister (s2)',
 'sarjana (s1)',
 'sertifikat professional',
 'sma',
 'smu/smk/stm',
 'tidak terspesifikasi'}

In [20]:
education_level_dict = {'d3 (diploma)': 3,
                     'd4 (diploma)': 3,
                     'diploma pascasarjana': 3,
                     'doktor (s3)': 5,
                     'gelar professional': 3,
                     'magister (s2)': 4,
                     'sarjana (s1)': 3,
                     'sertifikat professional': 2,
                     'sma': 2,
                     'smu/smk/stm': 2,
                     'tidak terspesifikasi': 1
                  }


In [21]:
def f_education_level(x):
    x = x.lower()
    splitted = x.split(',')
    
    l = [education_level_dict[i.strip()] for i in splitted]
    
    return min(l), max(l)

# transform
train_cat['education_level_min'] = train_cat['education_level'].map(f_education_level).map(lambda t: t[0])
train_cat['education_level_max'] = train_cat['education_level'].map(f_education_level).map(lambda t: t[1])
del train_cat['education_level']

test_cat['education_level_min'] = test_cat['education_level'].map(f_education_level).map(lambda t: t[0])
test_cat['education_level_max'] = test_cat['education_level'].map(f_education_level).map(lambda t: t[1])
del test_cat['education_level']

#### job_function

In [22]:
def f_job_function(x):
    x = x.strip().lower()
    splitted = x.split(',')
    
    return (splitted[0], splitted[-1])

In [23]:
train_cat['job_function_broad'] = train_cat['job_function'].map(f_job_function).map(lambda x: x[0])
train_cat['job_function_specific'] = train_cat['job_function'].map(f_job_function).map(lambda x: x[1])
del train_cat['job_function']

In [24]:
test_cat['job_function_broad'] = test_cat['job_function'].map(f_job_function).map(lambda x: x[0])
test_cat['job_function_specific'] = test_cat['job_function'].map(f_job_function).map(lambda x: x[1])
del test_cat['job_function']

#### company_industry

In [25]:
company_industry_ref = train_df.groupby('company_industry').agg({'salary':np.mean})
company_industry_ref = company_industry_ref.reset_index()

In [26]:
# transform train
company_industry_input = train_cat[['company_industry']].merge(company_industry_ref, how='left', on='company_industry')['salary']
train_cat['company_industry'] = company_industry_input

In [27]:
# transform test
company_industry_input_ts = test_cat[['company_industry']].merge(company_industry_ref, how='left', on='company_industry')['salary']
test_cat['company_industry'] = company_industry_input_ts


# Data Preprocessing

In [28]:
train_cat.head()

,location,career_level,company_industry,education_level_min,education_level_max,job_function_broad,job_function_specific
0,Bandung,4,NaN,2,3,manufaktur,pemeliharaan
1,Jakarta Raya,4,1.228130e+07,3,4,manufaktur,pembelian/manajemen material
2,Jakarta Barat,3,6.233821e+06,3,3,penjualan / pemasaran,penjualan ritel
3,Cirebon,3,6.300965e+06,3,4,pelayanan,logistik/rantai pasokan
4,Bekasi,2,1.228130e+07,2,3,lainnya,jurnalis/editor


#### company_industry

In [44]:
industry_salary_ref = train_df[['company_industry', 'salary']].groupby('company_industry').agg({'salary':np.mean}).reset_index()

# transform train
train_cat['company_industry'] = train_cat[['company_industry']].merge(industry_salary_ref, how='left', on='company_industry')['salary']

# transform test
test_cat['company_industry'] = test_cat[['company_industry']].merge(industry_salary_ref, how='left', on='company_industry')['salary']

#### job_function

In [30]:
function_salary_ref = pd.concat([train_cat[['job_function_broad']], train_df[['salary']]],
                                axis=1)

function_salary_ref = function_salary_ref.groupby('job_function_broad').agg({'salary':np.mean}).reset_index()


# transform train
train_cat['job_function_broad'] = train_cat[['job_function_broad']].merge(function_salary_ref, how='left', on='job_function_broad')['salary']

# transform test
test_cat['job_function_broad'] = test_cat[['job_function_broad']].merge(function_salary_ref, how='left', on='job_function_broad')['salary']


#### delete some columns

In [35]:
train_cat.drop(['location', 'job_function_specific'], axis=1, inplace=True)
test_cat.drop(['location', 'job_function_specific'], axis=1, inplace=True)

#### combine all cols again

In [46]:
train_pp = pd.concat([train_num, train_cat, train_df[['id', 'salary']]], axis=1)
test_pp = pd.concat([test_num, test_cat], axis=1)

#### delete some outlier salaries on train

In [53]:
delete_ixes = train_pp.loc[train_pp['id'].isin([163,2822,8624,15076,20909,30896]), :].index
train_pp.drop(delete_ixes, axis=0, inplace=True)

In [54]:
del train_pp['id']

# Modelling

### fill up nan salaries in train

In [57]:
train_pp_tr = train_pp.loc[~train_pp['salary'].isna(), :]
train_pp_ts = train_pp.loc[train_pp['salary'].isna(), :]

In [58]:
train_pp_tr_X = train_pp_tr.copy().drop('salary', axis=1)
train_pp_tr_y = train_pp_tr.copy()['salary']

In [63]:
# XGB Regressor - Semi Supervised
xgb = XGBRegressor(n_estimators=300, objective='reg:squarederror', random_state=69)
xgb.fit(train_pp_tr_X, train_pp_tr_y)
val = cross_val_score(xgb, train_pp_tr_X, train_pp_tr_y, scoring='neg_root_mean_squared_error', cv=5)
print(np.mean(val*-1))

4090946.7337979553


In [69]:
filled_salary = xgb.predict(train_pp_ts.drop(['salary'], axis=1))
train_pp_ts['salary'] = filled_salary

<ipython-input-69-f7496d433094>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_pp_ts['salary'] = filled_salary


In [70]:
# combine train again
train_pp = pd.concat([train_pp_tr, train_pp_ts], axis=0)

### predict on test

In [71]:
train_pp_X = train_pp.drop('salary', axis=1)
train_pp_y = train_pp['salary']

In [72]:
# XGB Regressor - Semi Supervised
xgb = XGBRegressor(n_estimators=300, objective='reg:squarederror', random_state=69)
xgb.fit(train_pp_X, train_pp_y)
val = cross_val_score(xgb, train_pp_X, train_pp_y, scoring='neg_root_mean_squared_error', cv=5)
print(np.mean(val*-1))

1217386.4636238662


In [78]:
salary_pred = xgb.predict(test_pp)

In [88]:
sample_sub['salary'] = salary_pred
sample_sub['id'] = test_df['id']

In [89]:
sample_sub.to_csv('xgb basic.csv', index=False)